In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
merged = pd.read_csv("Wide_Simple_V1.csv")

In [3]:
merged['Date/Time'] = pd.to_datetime(merged["Date/Time"], format='%Y-%m-%d %H:%M:%S')

In [4]:
merged = merged.sort_values(by='Date/Time', ascending=False)
merged = merged.reset_index(drop=True)

In [5]:
merged['date_minus_1year'] = merged['Date/Time'].apply(lambda x: x - pd.DateOffset(years=1))

## New Features

In [6]:
deb = time.time()

# Date when we stop computing the features (one year later than the earliest date)
min_plus_1year = pd.to_datetime('17.06.1995',format='%d.%m.%Y')

nb_matches1, nb_matches2 = [],[]

goals_past1, goals_past2 = [],[]
ratio_goals_past1, ratio_goals_past2 = [],[]

nb_wins1, nb_wins2 = [],[]
ratio_wins1, ratio_wins2 = [],[]
nb_losses1, nb_losses2 = [],[]
ratio_losses1, ratio_losses2 = [],[]
nb_draws1, nb_draws2 = [],[]
ratio_draws1, ratio_draws2 = [],[]

nb_wins_opp1, nb_wins_opp2 = [],[]
ratio_wins_opp1, ratio_wins_opp2 = [],[]
nb_losses_opp1, nb_losses_opp2 = [],[]
ratio_losses_opp1, ratio_losses_opp2 = [],[]
nb_draws_opp1, nb_draws_opp2 = [],[]
ratio_draws_opp1, ratio_draws_opp2 = [],[]

for i in range(len(merged)):
    if merged['Date/Time'][i] >= min_plus_1year: # One year so that we can compute the features for all the other dates
        
        # Initialize our variables
        mat1, mat2 = 0,0  
        g1,g2, g1_tot,g2_tot = 0,0,0,0
        w1,w2, l1,l2, d1,d2, tot1,tot2 = 0,0,0,0,0,0,0,0
        w1_opp,w2_opp, l1_opp,l2_opp, d1_opp,d2_opp, tot_opp = 0,0,0,0,0,0,0
    
        # Only consider the sub-dataframe between the given date and one year earlier
        idx = list(np.array((merged.loc[merged['Date/Time'] < merged['Date/Time'][i]].index) & 
                            (merged.loc[merged['Date/Time'] >= merged['date_minus_1year'][i]].index)))
        p = merged.iloc[idx]
        p = p.reset_index(drop=True)
    
        # Iterate over the sub-dataframe
        for j in range(len(p)):
    
            # Number of played matches
        
            if (merged["Team1"][i] == p["Team1"][j] or merged["Team1"][i] == p["Team2"][j]):
                mat1 += 1
            if (merged["Team2"][i] == p["Team1"][j] or merged["Team2"][i] == p["Team2"][j]):
                mat2 += 1

                
            # (Relative) number of goals
            
            if merged["Team1"][i] == p["Team1"][j]:
                g1 += p["Score1"][j]
                g1_tot += p["Score1"][j]+p["Score2"][j]
            elif merged["Team1"][i] == p["Team2"][j]:
                g1 += p["Score2"][j]
                g1_tot += p["Score1"][j]+p["Score2"][j]

            if merged["Team2"][i] == p["Team1"][j]:
                g2 += p["Score1"][j]
                g2_tot += p["Score1"][j]+p["Score2"][j]
            elif merged["Team2"][i] == p["Team2"][j]:
                g2 += p["Score2"][j]
                g2_tot += p["Score1"][j]+p["Score2"][j]

                
            # (Relative) number of wins / losses / draws (in general)
            
            if merged["Team1"][i] == p["Team1"][j]:
                tot1 += 1
                if p['Score1'][j] > p['Score2'][j]:
                    w1 += 1
                elif p['Score1'][j] < p['Score2'][j]:
                    l1 += 1
                else:
                    d1 += 1
            elif merged["Team1"][i] == p["Team2"][j]:
                tot1 += 1
                if p['Score2'][j] > p['Score1'][j]:
                    w1 += 1
                elif p['Score2'][j] < p['Score1'][j]:
                    l1 += 1
                else:
                    d1 += 1
                    
            if merged["Team2"][i] == p["Team1"][j]:
                tot2 += 1
                if p['Score1'][j] > p['Score2'][j]:
                    w2 += 1
                elif p['Score1'][j] < p['Score2'][j]:
                    l2 += 1
                else:
                    d2 += 1
            elif merged["Team2"][i] == p["Team2"][j]:
                tot2 += 1
                if p['Score2'][j] > p['Score1'][j]:
                    w2 += 1
                elif p['Score2'][j] < p['Score1'][j]:
                    l2 += 1
                else:
                    d2 += 1
                    
                    
            # (Relative) number of wins / losses / draws (against opponent)

            if merged["Team1"][i] == p["Team1"][j] and merged["Team2"][i] == p["Team2"][j]:
                tot_opp += 1
                if p['Score1'][j] > p['Score2'][j]:
                    w1_opp += 1
                    l2_opp += 1
                elif p['Score1'][j] > p['Score2'][j]:
                    l1_opp += 1
                    w2_opp += 1
                else:
                    d1_opp += 1
                    d2_opp += 1
            elif merged["Team1"][i] == p["Team2"][j] and merged["Team2"][i] == p["Team1"][j]:
                tot_opp += 1
                if p['Score2'][j] > p['Score1'][j]:
                    w1_opp += 1
                    l2_opp += 1
                elif p['Score2'][j] < p['Score1'][j]:
                    l1_opp += 1
                    w2_opp += 1
                else:
                    d1_opp += 1
                    d2_opp += 1
    
        # Add our values
        nb_matches1.append(mat1)
        nb_matches2.append(mat2)
        
        goals_past1.append(g1)
        goals_past2.append(g2)

        if g1_tot == 0:
            ratio_goals_past1.append(0)
        else:
            ratio_goals_past1.append(g1/g1_tot)
        if g2_tot == 0:
            ratio_goals_past2.append(0)
        else:
            ratio_goals_past2.append(g2/g2_tot)

        nb_wins1.append(w1)
        nb_wins2.append(w2)
        nb_losses1.append(w1)
        nb_losses2.append(w2)
        nb_draws1.append(d1)
        nb_draws2.append(d2)

        if tot1 == 0:
            ratio_wins1.append(0)
            ratio_losses1.append(0)
            ratio_draws1.append(0)
        else:
            ratio_wins1.append(w1/tot1)
            ratio_losses1.append(l1/tot1)
            ratio_draws1.append(d1/tot1)
        if tot2 == 0:
            ratio_wins2.append(0)
            ratio_losses2.append(0)
            ratio_draws2.append(0)
        else:
            ratio_wins2.append(w2/tot2)
            ratio_losses2.append(l2/tot2)
            ratio_draws2.append(d2/tot2)

        nb_wins_opp1.append(w1_opp)
        nb_wins_opp2.append(w2_opp)
        nb_losses_opp1.append(l1_opp)
        nb_losses_opp2.append(l2_opp)
        nb_draws_opp1.append(d1_opp)
        nb_draws_opp2.append(d2_opp)

        if tot_opp == 0:
            ratio_wins_opp1.append(0)
            ratio_losses_opp1.append(0)
            ratio_draws_opp1.append(0)
            ratio_wins_opp2.append(0)
            ratio_losses_opp2.append(0)
            ratio_draws_opp2.append(0)
        else:
            ratio_wins_opp1.append(w1_opp/tot_opp)
            ratio_losses_opp1.append(l1_opp/tot_opp)
            ratio_draws_opp1.append(d1_opp/tot_opp)
            ratio_wins_opp2.append(w2_opp/tot_opp)
            ratio_losses_opp2.append(l2_opp/tot_opp)
            ratio_draws_opp2.append(d2_opp/tot_opp)

        if i%500 == 0:
            print(i)
    
end = time.time()
print((end-deb) / 60," minutes")

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
43.04453285535177  minutes


In [7]:
idx = list(np.array(merged.loc[merged['Date/Time'] >= min_plus_1year].index))
result = merged.iloc[idx]
result = result.reset_index(drop=True)

In [8]:
result["nb_matches1"] = nb_matches1
result["nb_matches2"] = nb_matches2

result["goals_past1"] = goals_past1
result["goals_past2"] = goals_past2
result["ratio_goals_past1"] = ratio_goals_past1
result["ratio_goals_past2"] = ratio_goals_past2

result["nb_wins1"] = nb_wins1
result["nb_wins2"] = nb_wins2
result["ratio_wins1"] = ratio_wins1
result["ratio_wins2"] = ratio_wins2

result["nb_losses1"] = nb_losses1
result["nb_losses2"] = nb_losses2
result["ratio_losses1"] = ratio_losses1
result["ratio_losses2"] = ratio_losses2

result["nb_draws1"] = nb_draws1
result["nb_draws2"] = nb_draws2
result["ratio_draws1"] = ratio_draws1
result["ratio_draws2"] = ratio_draws2

result["nb_wins_opp1"] = nb_wins_opp1
result["nb_wins_opp2"] = nb_wins_opp2
result["ratio_wins_opp1"] = ratio_wins_opp1
result["ratio_wins_opp2"] = ratio_wins_opp2

result["nb_losses_opp1"] = nb_losses_opp1
result["nb_losses_opp2"] = nb_losses_opp2
result["ratio_losses_opp1"] = ratio_losses_opp1
result["ratio_losses_opp2"] = ratio_losses_opp2

result["nb_draws_opp1"] = nb_draws_opp1
result["nb_draws_opp2"] = nb_draws_opp2
result["ratio_draws_opp1"] = ratio_draws_opp1
result["ratio_draws_opp2"] = ratio_draws_opp2

In [9]:
result = result.drop(['date_minus_1year'],axis=1)

In [10]:
result.to_csv("Wide_WithComputedFeatures_V2.csv", index=False, encoding="utf-8")

In [11]:
len(merged)

9127

In [12]:
len(result)

9075